In [1]:
%matplotlib inline
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
from IPython.display import Image
from IPython.core.display import HTML 

import sys
sys.path.append('./cocoapi/PythonAPI')
from pycocotools.coco import COCO

In [2]:
coco = COCO('./data/foilv1.0_train_2017.json')

loading annotations into memory...
Done (t=1.00s)
creating index...
index created!


In [3]:
# Get all dataset annotations
anns = coco.loadAnns(coco.getAnnIds())
print(anns[:5])

[{'id': 789366, 'foil_id': 2000003, 'image_id': 57870, 'caption': 'a long restaurant couch with rattan rounded back chairs.', 'target_word': '', 'foil_word': '', 'foil': True}, {'id': 789366, 'foil_id': 2000003, 'image_id': 57870, 'caption': 'a long restaurant couch with rattan rounded back chairs.', 'target_word': '', 'foil_word': '', 'foil': True}, {'id': 789888, 'foil_id': 2000006, 'image_id': 57870, 'caption': 'a long couch with a plant on top of it surrounded with wooden chairs', 'target_word': '', 'foil_word': '', 'foil': True}, {'id': 789888, 'foil_id': 2000006, 'image_id': 57870, 'caption': 'a long couch with a plant on top of it surrounded with wooden chairs', 'target_word': '', 'foil_word': '', 'foil': True}, {'id': 791316, 'foil_id': 2000008, 'image_id': 57870, 'caption': 'a long couch with a flower arrangement in the middle for meetings', 'target_word': '', 'foil_word': '', 'foil': True}]


In [4]:
def display_img(img_ids, caption=''):
    imgs = coco.loadImgs(img_ids)
    for img in imgs:
        display(Image(url=img['flickr_url'], width=100, height=100))
        display(caption)
        

In [5]:
# Input can be 'id', foil_id, 'image_id'
def get_unique_ids(id_type):
    anns = coco.loadAnns(coco.getAnnIds())
    ids = {ann[id_type] for ann in anns}
    return list(ids)

In [6]:
# There are more foil ids than img ids since there are multiple ways of foiling one image
print("# Foils: ", len(get_unique_ids('foil_id')))
print("# Images: ", len(get_unique_ids('image_id')))

# Foils:  197788
# Images:  65697


In [7]:
# Look at captions for the same image
imgs = get_unique_ids('image_id')
img_0 = imgs[11]
captions_0 = {(ann['caption'], ann['foil']) for ann in anns if ann['image_id'] == img_0}
display_img(img_0, captions_0)

{('a cat on a surfboard is riding in on the wake.', True),
 ('a cat riding a surfboard in the ocean.', True),
 ('a cat surfing on a surfboard on a body of water.', True),
 ('a cow laying on a surf board and riding a small wave', True)}